<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/gazette/UK%20Gazette.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies
!pip install neo4j beautifulsoup4 spacy pandas
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.0/171.0 KB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
Discarding https://files.pythonhosted.org/packages/d3/b7/f1f8ffdb12d7e10d31b0a23bc2d8a78ad4659474eda2a62f023717df2ba1/neo4j-5.5.0.tar.gz#sha256=2632386380b2ebb7d6a80e4186899ef342ef0507601c65e200696f13742046b8 (from https://pypi.org/simple/neo4j/) (requires-python:>=3.7): Requested unknown from https://files.pythonhosted.org/packages/d3/b7/f1f8ffdb12d7e10d31b0a23bc2d8a78ad4659474eda2a62f023717df2ba1/neo4j-5.5.0.tar.gz#sha256=2632386380b2ebb7d6a80e4186899ef342ef0507601c65e200696f13742046b8 has inconsistent name: filename has 'neo4j', but metadata has 'unknown'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.5/162.5 KB 18.0 MB/s eta 0:00

# Update

* Updates to GDS 2.3.0 and Neo4j v5

# Represent United Kingdom's public record as a knowledge graph
## Utilize UK Gazette API capabilities to construct a knowledge graph and analyze it in Neo4j
I love constructing knowledge graphs from various sources. I've wanted to create a government knowledge graph for some time now but was struggling to find any data that is easily accessible and doesn't require me to spend weeks developing a data pipeline. At first, I thought I would have to use OCR and NLP techniques to extract valuable information from public records, but luckily I stumbled upon UK Gazette. The UK Gazette is a website that holds the United Kingdom's official public record information. All the content on the website and via its APIs is available under the Open Government License v3.0. Since they offer the public record information through an API endpoint, we don't have to use any scraping tool to extract the information. Even more impressive is that you can export the data in linked data format (RDF). Linked data is a format to represent structured data which is interlinked and essentially represents a graph as you are dealing with nodes and relationships.

Since the linked data structure (RDF) already contains information about nodes and relationships, we don't have to define a graph schema manually.
Most graph databases use either the RDF (Resource Description Framework) or the LPG (Labeled-property graph) model under the hood. If you are using an RDF graph database, the structure of the linked data information will be identical to the graph model in the database. However, as you might know from my previous posts, I like to use Neo4j, which utilizes an LPG graph model. I won't go much into the difference between the two models here. If you want to learn more about the difference between the RDF and LPG models, I would point you to the [presentation by Jesús Barrasa](https://neo4j.com/blog/rdf-triple-store-vs-labeled-property-graph-difference/).
Since the linked data structure is frequently used to transmit data, the folks at Neo4j have made it easy to import or export data in the linked data format by using the [Neosemantics library](https://neo4j.com/labs/neosemantics/). In this post, we will be using the Neo4j database in combination with the Neosemantics library to store the linked data information fetched from the UK Gazette's API.
## Environment setup
To follow along, you need to have a running instance of the Neo4j database with the Neosemantics library installed.
One option is to use the Neo4j Sandbox environment, a free cloud instance of the Neo4j database with the Neosemantics library pre-installed. If you want to use the Neo4j Sandbox environment, [start a blank project](https://sandbox.neo4j.com/?usecase=blank-sandbox) that comes with an empty database.
On the other hand, you could also use a local environment of Neo4j. If you opt for a local version, I recommend using the Neo4j Desktop application, a database management application that has a simple interface for adding plugins with a single click.
## Setting up connection to Neo4j instance
Before we begin, we have to establish connection with Neo4j from the notebook environment. If you are using the Sandbox instance, you can copy details from the Connection Details tab.

In [2]:
# Define Neo4j connections
import pandas as pd
from neo4j import GraphDatabase
host = 'bolt://3.231.25.240:7687'
user = 'neo4j'
password = 'hatchets-visitor-axes'
driver = GraphDatabase.driver(host,auth=(user, password))

def run_query(query, params={}):
    with driver.session() as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

## Configuring Neosemantics library
It is required to define a unique constraint on the Resource nodes for the Neosemantics library to work. You can define the unique constraint using the following Cypher statement.

In [3]:
run_query("""
CREATE CONSTRAINT n10s_unique_uri IF NOT EXISTS FOR (r:Resource)
REQUIRE r.uri IS UNIQUE
""")

""


Next, we need to define the Neosemantics configuration. We have a couple of options to specify how we want the RDF data to be imported as an LPG graph. We'll keep most of the configuration default and only set the handleVocabUri and applyNeo4jNaming parameters. Again, you can inspect the documentation for the [complete reference of configuration options](https://neo4j.com/labs/neosemantics/4.3/reference/).

Use the following Cypher statement to define the Neosemantics configuration.

In [4]:
run_query("""
CALL n10s.graphconfig.init({
  handleVocabUris: 'MAP',
  applyNeo4jNaming: true
})
""")

,param,value
0,handleVocabUris,MAP
1,handleMultival,OVERWRITE
2,handleRDFTypes,LABELS
3,keepLangTag,False
4,keepCustomDataTypes,False
5,applyNeo4jNaming,True
6,baseSchemaNamespace,neo4j://graph.schema#
7,baseSchemaPrefix,n4sch
8,classLabel,Class
9,subClassOfRel,SCO


## Construct a knowledge graph of UK public record
We will utilize the UK Gazette API to search for notices. The Notice Feed API is publicly available and doesn't require any authorization. However, you need to pretend you are a browser for it to work for some reason. I have no idea the reason behind this, but I spent 30 minutes of my life trying to make it work. The documentation of the API is available on [GitHub](https://github.com/TheGazette/DevDocs/blob/master/notice/notice-feed.md).

The main two parameters to filter notices via the API are the category code and notice type. The category code is the higher-level filter, while the notice type allows you to select only a subsection of a category. The complete list of category codes and notice types is available on the following website. There is a broad selection of notices you can choose from, ranging from State and Parliament to Companies regulation and more.
As mentioned, we can download the linked data format information for each notice. A nice thing about the Neosemantics library is that it can fetch data from local files as well as simple APIs. The workflow will be the following.
Use the Notice Feed API to find relevant notice ids
Use the Neosemantics to extract RDF information about specified notice ids and store it in Neo4j.

Lastly, we will define the function that will take in the category code and notice type parameters and store the information about notices in the Neo4j database.

In [5]:
import requests
from requests.structures import CaseInsensitiveDict

# Query to import RDF/XML data to Neo4j using Neosemantics
import_rdf_query = """
UNWIND $data AS link
CALL n10s.rdf.import.fetch(
  link,
  'RDF/XML'
) YIELD triplesLoaded
RETURN sum(triplesLoaded) AS totalTriplesLoaded
"""

def make_request(uri):
    # For some reason, the API only works when I pretend to be a browser
    headers = CaseInsensitiveDict()
    headers["user-agent"] = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"
    return requests.get(uri, headers=headers)
    
def ukgazzette_to_neo4j(pages=1, categorycode="", noticetype=""):
    for page in range(1, pages + 1):
        baseUrl = f"https://www.thegazette.co.uk/all-notices/notice/data.json"
        ccode = "categorycode=" + categorycode + "&" if categorycode else ""
        ntype = "noticetype=" + noticetype + "&" if noticetype else ""
        parameters = f"?{ccode}{ntype}results-page-size=100&sort-by=latest-date&results-page={page}"
        
        try:
            response = make_request(baseUrl + parameters)
            responseJson = response.json()
        except Exception as e:
            print(response.text)
            print(e)
            break

        # Define RDF/XML URL links
        data = []
        for notice in responseJson['entry']:
            id = notice['id'].split('/')[-1]
            rdf_uri = f"https://www.thegazette.co.uk/notice/{id}/data.rdf?view=linked-data"
            data.append(rdf_uri)

        # Import RDF into Neo4j with Neosemantics
        query_response = run_query(import_rdf_query, {'data': data})
        print(query_response)

The Cypher statement expects the data parameter to contain a list of links where the RDF/XML information about the notices is available. The Neosemantics library supports other RDF serialization formats as well, such as JSON-LD and others, if you might be wondering.

We get 100 notice ids for each request to the Notice Feed API. I've included the pagination feature in the function if you want to import more. As you might see from the code, we make a request to the notice feed and construct a list of links where the RDF/XML information about notices is stored. Next, we input that list as the parameter to the Cypher statement, where the Neosemantics library will iterate over all the links and store the information in Neo4j. It's about as simple as it gets.
Now we can go ahead and import the last 1000 notices under the state category code. If you look at the notice code reference, you can see that the state notices fall under the category code value of 11.

In [6]:
# Import last 1000 state notices
ukgazzette_to_neo4j(10, "11")

   totalTriplesLoaded
0                4656
   totalTriplesLoaded
0                5129
   totalTriplesLoaded
0                5129
   totalTriplesLoaded
0                5157
   totalTriplesLoaded
0                5340
   totalTriplesLoaded
0                5105
   totalTriplesLoaded
0                5101
   totalTriplesLoaded
0                5060
   totalTriplesLoaded
0                5022
   totalTriplesLoaded
0                5116


The graph schema of the imported graph is complex and not easy to visualize so we will skip that. I didn't spend much time apprehending the whole graph structure, but I prepared a couple of sample Cypher statements that could get us started.

For example, we can examine the last five receivers of any awards.

In [7]:
# Who has received any awards

run_query("""
MATCH (award)<-[:ISAWARDED]-(t:AwardandHonourThing)-[:HASAWARDEE]->(person)-[:HASEMPLOYMENT]->(employment)-[:ISMEMBEROFORGANISATION]->(organization)
RETURN award.label AS award,
       t.relatedDate AS relatedDate,
       person.name AS person,
       employment.jobTitle AS jobTitle,
       organization.name AS organization
ORDER BY relatedDate DESC
LIMIT 5
""")


,award,relatedDate,person,jobTitle,organization
0,K.P.M.,2022-12-31,Lindsay Leanne FISHER,None,None
1,K.A.M.,2022-12-31,Edward Michael Rhodri O’BRIAN,None,None
2,K.F.S.M.,2022-12-31,Robert STRANG,None,None
3,K.F.S.M.,2022-12-31,John ROBERTS,None,None
4,K.P.M.,2022-12-31,Alan TODD,None,None


Remember, these was the example I showed in the introduction of this article. I've learned that one can also be appointed as the commander in the Order of the British Empire.

In [8]:
run_query("""
MATCH (n:CommanderOrderOfTheBritishEmpire)<-[:ISAPPOINTEDAS]-(notice)-[:HASAPPOINTEE]->(appointee),
      (notice)-[:HASAUTHORITY]->(authority)
RETURN n.label AS award,
       notice.relatedDate AS date,
       appointee.name AS appointee,
       authority.label AS authority
ORDER BY date DESC
LIMIT 5
""")

,award,date,appointee,authority


To steer away from awards, we can also inspect which notices that are related to various legislation.

In [9]:
run_query("""
MATCH (provenance)<-[:HAS_PROVENANCE]-(n:Notice)-[:ISABOUT]->(l:Legislation:NotifiableThing)-[:RELATEDLEGISLATION]->(related)
RETURN n.hasNoticeID AS noticeID,
       n.uri AS noticeURI,
       l.relatedDate AS date,
       provenance.uri AS provenance,
       collect(related.label) AS relatedLegislations
ORDER BY date DESC
LIMIT 5
""")

,noticeID,noticeURI,date,provenance,relatedLegislations


The nice thing about our knowledge graph is that it contains all the data references to the Gazette website. This allows us to verify and also find more information if needed. In addition, through my data exploration, I've noticed that not all information is parsed from notices as a lot of information is hard to structure as a graph automatically. More on that later.
Suppose you are like me and get quickly bored by state information. In that case, you could fetch more business-related information such as companies buying back their own stock, company directors being disqualified, or partnership dissolutions.

In [10]:
# Redemption or purchase of own shares out of capital, Company director disqualification order, Dissolution of partnership
ukgazzette_to_neo4j(10, "26+27", "2602+2608+2702")

   totalTriplesLoaded
0                4100
   totalTriplesLoaded
0                3883
   totalTriplesLoaded
0                3945
   totalTriplesLoaded
0                4025
   totalTriplesLoaded
0                4127
   totalTriplesLoaded
0                4517
   totalTriplesLoaded
0                4023
   totalTriplesLoaded
0                3872
   totalTriplesLoaded
0                3745
   totalTriplesLoaded
0                4242


I've spent 30 minutes figuring out how to properly use notice types and category code parameters to filter notice feeds. You must also include the category code parameter when you want to filter by notice type. Otherwise, the filtering won't work as expected.
We don't have to worry about creating separate graphs or databases for additional notice feeds. The graph schema is already defined in the RDF/XML data structure, and you can import all the notice types into a single Neo4j instance.

Now you can examine which partnerships have dissolved.

In [11]:
run_query("""
MATCH (n:PartnershipDissolutionNotice)-[:ISABOUT]->(notifiableThing)-[:HASCOMPANY]->(partnership),
      (notifiableThing)-[:ISENABLEDBYLEGISLATION]->(enabledby)
RETURN n.hasNoticeID AS noticeID,
       notifiableThing.relatedDate AS date,
       notifiableThing.uri AS noticeURI,
       enabledby.label AS enablingLegislation,
       partnership.name AS partnership
ORDER BY date DESC
LIMIT 5
""")

,noticeID,date,noticeURI,enablingLegislation,partnership
0,4269576,2023-01-31,https://www.thegazette.co.uk/id/notice/4269576...,LIMITED PARTNERSHIPS ACT 1907,AF VII LP
1,4269885,2023-01-31,https://www.thegazette.co.uk/id/notice/4269885...,LIMITED PARTNERSHIPS ACT 1907,Viking DS LP
2,4269565,2023-01-24,https://www.thegazette.co.uk/id/notice/4269565...,LIMITED PARTNERSHIPS ACT 1907,LEHMAN BROTHERS HOLDINGS SCOTTISH LP 3
3,4258085,2023-01-04,https://www.thegazette.co.uk/id/notice/4258085...,LIMITED PARTNERSHIPS ACT 1907,ALPINA PARTNERS (SCOTGP) LP
4,4249626,2022-12-30,https://www.thegazette.co.uk/id/notice/4249626...,LIMITED PARTNERSHIPS ACT 1907,RANCH CAPITAL SLP


Another interesting information is about which companies have or intend to buyback their own shares.

In [12]:
run_query("""
MATCH (legislation)<-[:RELATEDLEGISLATION]-(n:RedemptionOrPurchase)-[:HASCOMPANY]->(company)
RETURN n.relatedDate AS date,
       company.name AS company,
       company.uri AS companyURI,
       collect(legislation.label) AS relatedLegislations,
       n.uri AS noticeURI
ORDER BY date DESC
LIMIT 5
""")

,date,company,companyURI,relatedLegislations,noticeURI
0,2022-11-22,TREASURYSPRING LIMITED\n\n\n,http://business.data.gov.uk/id/company/10373508,"[Companies Act 2006, s. 719, COMPANIES ACT 2006]",https://www.thegazette.co.uk/id/notice/4215745...
1,2022-09-23,\n EXPRESS HEATING CO. LIMITED,http://business.data.gov.uk/id/company/SC046408,"[COMPANIES ACT 2006, Companies Act 2006, s. 719]",https://www.thegazette.co.uk/id/notice/4166832...


# Taking in to the next level
As mentioned before, there are some example where not all information is extracted from notices in a linked data structure. One such example are the members change in partnership. We have the information about the partnership in which the membership changed, but not exactly what has changed. All the data we can retrieve is the following:

In [13]:
run_query("""
MATCH (notice)-[:ISABOUT]->(n:PartnershipChangeInMembers)-[:HASCOMPANY]->(company)
RETURN notice.hasNoticeID AS noticeID,
       notice.uri AS noticeURI,
       n.relatedDate AS date,
       company.name AS company
ORDER BY date DESC
LIMIT 5
""")

,noticeID,noticeURI,date,company
0,4255004,https://www.thegazette.co.uk/id/notice/4255004,2023-01-10,\n ACTONS SOLICITORS
1,4255003,https://www.thegazette.co.uk/id/notice/4255003,2023-01-10,\n ACTONS SOLICITORS
2,4256953,https://www.thegazette.co.uk/id/notice/4256953,2023-01-01,\nESPRIT CAPITAL I FUND NO.2 LIMITED PARTNERSHIP
3,4256950,https://www.thegazette.co.uk/id/notice/4256950,2023-01-01,ESPRIT CAPITAL I FUND NO.1 LIMITED PARTNERSHIP
4,4247940,https://www.thegazette.co.uk/id/notice/4247940,2022-12-23,GLG PARTNERS LP\n


For example, if we inspect the first notice on the website we can observe that a the actual changes are not available in the linked data format.

I totally understand why the actual changes are not in a structured format. The reason is that there are too many variations of the membership change notice to capture them all in a structured way.

It seems like all the roads lead to Rome, or in our case, when dealing with text, you will possibly have to utilize NLP techniques. So I've added a simple example of using SpaCy to extract organizations and person entities from notices.

In [14]:
from bs4 import BeautifulSoup as bs
import spacy

nlp = spacy.load("en_core_web_sm")

def extract_entities(noticeId):
    print(f"\nExtracting entities for {noticeId}")
    uri = f"https://www.thegazette.co.uk/notice/{noticeId}/data.xml?download=true"
    content = make_request(uri).content
    bs_content = bs(content, "lxml")
    text = " ".join([el.text for el in bs_content.findAll("p", {"data-gazettes":"Text"})])
    print(text)
    doc = nlp(text)
    # Find named entities, phrases and concepts
    print('Entities \n --------------------')
    for entity in doc.ents:
        if not entity.label_ in ['PERSON', 'ORG']:
            continue
        print(entity.text, entity.label_)


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


The text of notices is not stored in our knowledge graph, so we have to utilize the UK Gazette API to retrieve it. I've used BeatifulSoup to extract the text from the XML response and then run it through SpaCy's NLP pipeline to detect organizations and person mentions. The code doesn't store the entities back to Neo4j. I've just wanted to give you a simple example of how you could start utilizing NLP capabilities to extract more information.

We can now detect entities for a couple of changes in members of partnership notices.

In [15]:
partnership_changes = run_query("""
MATCH (notice)-[:ISABOUT]->(n:PartnershipChangeInMembers)
RETURN notice.hasNoticeID AS noticeID
LIMIT 5
""")['noticeID'].to_list()

for i in partnership_changes:
    extract_entities(i)


Extracting entities for 4270330
NOTICE IS HEREBY GIVEN of the Partnership heretofore subsisting between John Joseph Easter, Jonathan Norman Richards and Surbjit Kumar carrying on business as Solicitors at 1 Rose Hill, Willenhall, West Midlands, WV13 2AR under the style or firm of ROWLAND TILDESLEY & HARRIS has been dissolved by mutual consent as from the 31st January 2023 so far as concerns the said John Joseph Easter who retires from the firm and that all debts due to and owing by the said firm will be received and paid by the said continuing partners who will continue to carry on the said business under the same style or firm. Dated this 1st day of February 2023
Entities 
 --------------------
John Joseph Easter PERSON
Jonathan Norman Richards PERSON
Surbjit Kumar PERSON
Rose Hill PERSON
ROWLAND TILDESLEY & HARRIS ORG
John Joseph Easter PERSON

Extracting entities for 4263635

          Pursuant to section 10 of the Limited Partnerships Act 1907, notice is hereby given in respect of

The NLP pipeline doesn't extract specific changes, but at least it's a start since you can't create a rule-based difference extraction due to having a non-standard structure of the text. You can observe that even the two examples are wildly different in text structure and information.